In [8]:
import os
import re
import pandas as pd
from io import StringIO

In [56]:
# Specify the path to your .txt file
file_path = 'data/raw/lissy/france_gini.txt'


In [126]:
df = pd.read_fwf(file_path, skiprows=3, header=None, names=["region", "year", "avg_gini"])

In [115]:
df = df.reset_index()

In [127]:
df = df[['region', 'year', 'avg_gini']]

In [119]:
df.reset_index(drop=True, inplace=True)

In [122]:
print(df.to_string(index=False))

         region  year  avg_gini
         Alsace  1997     0.286
         Alsace  1998     0.273
         Alsace  1999     0.289
         Alsace  2000     0.299
         Alsace  2001     0.304
         Alsace  2002     0.307
         Alsace  2003     0.313
         Alsace  2004     0.287
         Alsace  2005     0.305
         Alsace  2006     0.319
         Alsace  2007     0.308
         Alsace  2008     0.310
         Alsace  2009     0.324
         Alsace  2010     0.321
         Alsace  2011     0.309
         Alsace  2012     0.309
         Alsace  2013     0.310
         Alsace  2014     0.300
         Alsace  2015     0.314
         Alsace  2016     0.307
         Alsace  2017     0.290
         Alsace  2018     0.314
      Aquitaine  1996     0.311
      Aquitaine  1997     0.312
      Aquitaine  1998     0.314
      Aquitaine  1999     0.304
      Aquitaine  2000     0.298
      Aquitaine  2001     0.295
      Aquitaine  2002     0.328
      Aquitaine  2003     0.329
      Aq

In [129]:
df.to_csv('data/cleaned/lissy/france_gini.csv', index=False)